In [47]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

%matplotlib inline
mpl.rcParams['figure.figsize'] = (16,9)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

## Data load

In [48]:
# try to parse the dates at the beginning
# it works out of the box if the date was stored ISO format YYYY-MM-DD

df_analyse=pd.read_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_small_flat_table.csv',sep=';',
                      parse_dates=[0])

df_analyse.sort_values('date',ascending=True).tail()

,date,Italy,US,Spain,Germany,"Korea,South"
208,2020-08-17,254235,5438325,359082,226700,0
209,2020-08-18,254636,5482416,364196,228120,0
210,2020-08-19,255278,5529824,370867,229706,0
211,2020-08-20,256118,5573847,377906,231292,0
212,2020-08-21,257065,5622540,386054,233029,0


## Helper Functions

In [49]:
def quick_plot(x_in, df_input, y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
        
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array
             array of date time object, or array of numbers
        df_input : pandas dataframe
             the plotting matrix where each column is plotted
             the name of the column will be used for the legend
        scale: str
             y-axis scale as 'log' or 'linear'
        slider: bool
             True or False for x-axis slider
             
             
        Returns:
        -------
        
    """
    fig = go.Figure()
    
    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                         x=x_in,
                         y=df_input[each],
                         name=each,
                         opacity=0.8))
        
    fig.update_layout(autosize=True,
            width=1024,
            height=768,
            font=dict(
                family="PT sans, monospace",
                size=18,
                color="#7f7f7f"
                         )
             )
        
    fig.update_yaxes(type=y_scale)#, range=[0.1,2]
    fig.update_xaxes(tickangle=-45,
                        nticks=20,
                        tickfont=dict(size=14,color="#7f7f7f")
                        )
    if slider==True:
            fig.update_layout(xaxis_rangeslider_visible=True)
            fig.show()
        

In [50]:
quick_plot(df_analyse.date,
           df_analyse.iloc[:,1:],
           y_scale='linear',
           slider=True)

In [51]:
threshold=100

In [52]:
country_list=['Italy',
             'US',
             'Spain',
             'Germany',
             'Korea,South',
             ]

In [53]:
compare_list=[]
for pos,country in enumerate(country_list):
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))

In [54]:
pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T

In [55]:
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

In [56]:
pd_sync_timelines.head()

,Italy,US,Spain,Germany,"Korea,South",date
0,155.0,104.0,120.0,130.0,NaN,0
1,229.0,174.0,165.0,159.0,NaN,1
2,322.0,222.0,222.0,196.0,NaN,2
3,453.0,337.0,259.0,262.0,NaN,3
4,655.0,451.0,400.0,482.0,NaN,4


In [57]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

$N(t)=N_0*2^{t/T}$

In [58]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [61]:
max_days=130

norm_slopes={
    'doubling every day':doubling_rate(100,np.arange(max_days),1),
    'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
}

In [62]:
pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines],axis=1)

,doubling every day,doubling every two days,doubling every 4 days,doubling every 10 days,Italy,US,Spain,Germany,"Korea,South",date
0,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,155.0,104.0,120.0,130.0,NaN,0
1,2.000000e+02,1.414214e+02,1.189207e+02,107.177346,229.0,174.0,165.0,159.0,NaN,1
2,4.000000e+02,2.000000e+02,1.414214e+02,114.869835,322.0,222.0,222.0,196.0,NaN,2
3,8.000000e+02,2.828427e+02,1.681793e+02,123.114441,453.0,337.0,259.0,262.0,NaN,3
4,1.600000e+03,4.000000e+02,2.000000e+02,131.950791,655.0,451.0,400.0,482.0,NaN,4
5,3.200000e+03,5.656854e+02,2.378414e+02,141.421356,888.0,519.0,500.0,670.0,NaN,5
6,6.400000e+03,8.000000e+02,2.828427e+02,151.571657,1128.0,711.0,673.0,799.0,NaN,6
7,1.280000e+04,1.131371e+03,3.363586e+02,162.450479,1694.0,1109.0,1073.0,1040.0,NaN,7
8,2.560000e+04,1.600000e+03,4.000000e+02,174.110113,2036.0,1561.0,1695.0,1176.0,NaN,8
9,5.120000e+04,2.262742e+03,4.756828e+02,186.606598,2502.0,2157.0,2277.0,1457.0,NaN,9


In [63]:
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines],axis=1)

In [16]:
quick_plot(pd_sync_timelines_w_slope.date,
          pd_sync_timelines_w_slope.iloc[:,0:5],
          y_scale='log',
          slider=True)

In [64]:
pd_sync_timelines_w_slope.to_csv('C:/Users/Nitin/ds-covid19/data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

## Understanding Linear Regression

In [17]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)

In [18]:
l_vec=len(df_analyse['Germany'])
X=np.arange(l_vec-5).reshape(-1,1)
y=np.log(np.array(df_analyse['Germany'][5:]))

In [19]:
reg.fit(X,y)

LinearRegression(fit_intercept=False)

In [20]:
X_hat=np.arange(l_vec).reshape(-1,1)
Y_hat=reg.predict(X_hat)

In [21]:
LR_inspect=df_analyse[['date','Germany']].copy()

In [22]:
LR_inspect['prediction']=np.exp(Y_hat)

In [23]:
quick_plot(LR_inspect.date,
          LR_inspect.iloc[:,1:],
          y_scale='log',
          slider=True)

## Doubling Rate - Piecewise Linear Regression

In [24]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)

In [25]:
from scipy import signal

In [27]:
country_list=['Italy',
             'US',
             'Spain',
             'Germany',
             'Korea,South',
             ]

for each in country_list:
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                          3, # window size used for filtering
                          1) # order of fitted polynomial

In [28]:
filter_cols=['Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter']
country_list=filter_cols

In [29]:
start_pos=5
quick_plot(df_analyse.date[start_pos:],
          df_analyse[filter_cols].iloc[start_pos:,:],
          y_scale='log',
          slider=True)

In [ ]:
reg = linear_model.LinearRegression(fit_intercept=True)
l_vec=len(df_analyse['US'])
X=np.arange(l_vec-50).reshape(-1,1)
y=np.array(df_analyse['US'][50:])

In [30]:
reg.fit(X,y)

LinearRegression()

In [31]:
reg.intercept_

-608343.548331589

In [32]:
reg.coef_

array([34023.63844415])

In [33]:
reg.coef_/reg.intercept_

array([-0.05592833])

In [36]:
def get_rate_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1,1)
    
    assert len(in_array)
    
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [40]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular,
     so https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [41]:
days_back = 3
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[each].rolling(window=days_back,
                              min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [43]:
df_analyse['Germany_DR_math']=df_analyse['Germany'].rolling(window=days_back,
                              min_periods=days_back).apply(doubling_time, raw=False)

In [42]:
days_back = 3
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[each].rolling(window=days_back,
                              min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [37]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1,1)
    
    assert len(in_array)
    
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [38]:
country_list=df_analyse.columns[1:]
for each in country_list:
    
    df_analyse[each+'_DR']=df_analyse[each].rolling(window=3,
                              min_periods=3).apply(get_rate_via_regression)

In [39]:
quick_plot(df_analyse.date,
           df_analyse.iloc[40:,[6,7,8,9,10]],
           y_scale='linear')

In [44]:
days_back = 3
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[each].rolling(window=days_back,
                              min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [45]:
df_analyse.columns

Index(['date', 'Italy', 'US', 'Spain', 'Germany', 'Korea,South',
       'Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter',
       'Korea,South_filter', 'Italy_DR', 'US_DR', 'Spain_DR', 'Germany_DR',
       'Korea,South_DR', 'Italy_filter_DR', 'US_filter_DR', 'Spain_filter_DR',
       'Germany_filter_DR', 'Korea,South_filter_DR', 'Germany_DR_math'],
      dtype='object')

In [46]:
start_pos=40
quick_plot(df_analyse.date[start_pos:],
           df_analyse.iloc[start_pos:,[11,12,13,14]],
           y_scale='linear',
           slider=True)